In [2]:
library(tidyverse)
library(tidycensus)
library(sf)

In [12]:
msa_code = "C3562" # NYC Metro
path = "/home/data/census/nyc/"
# census_api_key({key}, install = TRUE)

In [10]:
all_msa <- read_csv("https://www.bls.gov/cew/classifications/areas/qcew-county-msa-csa-crosswalk-csv.csv") # National MSA to county crosswalk
nyc_msa <- all_msa %>% filter(`MSA Code` == msa_code)
nyc_msa_counties <- nyc_msa %>% pull(`County Code`)

Rows: 3251 Columns: 7
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (7): County Code, County Title, MSA Code, MSA Title, MSA Type, CSA Code,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [13]:
tract_boundaries <- get_acs(geography = "tract",
                            state = c("NY", "CT", "PA", "NJ"),
                            year = 2020, 
                            cb = TRUE, 
                            geometry = T, 
                            variables = "B19013_001") %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% nyc_msa_counties) %>%
  select(-variable, -estimate, -moe)

tract_boundaries <- st_transform(tract_boundaries, "WGS84")


st_write(tract_boundaries, paste0(path, "geo/nyc_tracts.geojson"))

Getting data from the 2016-2020 5-year ACS


Fetching tract data by state and combining the result.



Writing layer `nyc_tracts' to data source 
  `/home/data/census/nyc/geo/nyc_tracts.geojson' using driver `GeoJSON'
Writing 5127 features with 3 fields and geometry type Multi Polygon.


In [14]:
block_group_boundaries <- get_acs(geography = "block group", 
                                  state = c("NY", "CT", "PA", "NJ"), 
                                  year = 2020, 
                                  cb = TRUE, 
                                  geometry = T,
                                  variables = "B19013_001") %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% nyc_msa_counties) %>%
  select(-variable, -estimate, -moe)

block_group_boundaries <- st_transform(block_group_boundaries, "WGS84")

st_write(block_group_boundaries, paste0(path, "geo/nyc_block_groups.geojson"))

Getting data from the 2016-2020 5-year ACS


Fetching block group data by state and combining the result.



  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
Writing layer `nyc_block_groups' to data source 
  `/home/data/census/nyc/geo/nyc_block_groups.geojson' using driver `GeoJSON'
Writing 15529 features with 3 fields and geometry type Multi Polygon.


In [19]:
# Income/Employment Variables

income_employment_vars <- c(
  "total_pop" = "B01001_001",
  "pop_in_poverty" = "B17001_002"
  # Add more
)

# Age/Disability Variables

age_disability_vars <- c(
  "male_under_18" = "B05003_003",
  "female_under_18" = "B05003_014",
  "total_over_65" = "B16007_014"
  # Add more
)

# Minority/Language Variables

race_language_vars <- c(
  "white_nh_pop" = "B03002_003",
  "black_nh_pop" = "B03002_004",
  "asian_nh_pop" = "B03002_006",
  "hispanic_pop" = "B03002_012"
  # Add more
)

# Housing/Transportation Variables

housing_transport_vars <- c(
  "total_households" = "B08201_001",
  "households_no_car" = "B08201_002"
  # Add more
)


In [20]:
acs_vars <- c(income_employment_vars, age_disability_vars, race_language_vars, housing_transport_vars)

acs_long <- get_acs(geography = "tract",
                            state = c("NY", "CT", "PA", "NJ"),
                            year = 2020, 
                            geometry = F, 
                            variables = acs_vars) %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% nyc_msa_counties) 

acs_wide <- acs_long %>%
  pivot_wider(id_cols = c(1,2), names_from = "variable", values_from = "estimate")


Getting data from the 2016-2020 5-year ACS

Fetching tract data by state and combining the result.



In [21]:
# Compute new variables

acs_wide <- acs_wide %>%
  mutate(total_under_18 = female_under_18 + male_under_18,
         )

write_csv(acs_wide, paste0(path, "acs/acs_wide.cvs"))
